In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import numpy as np
from PIL import Image
import pywt
import matplotlib.pyplot as plt
import seaborn as sns

# Custom FF++ dataset (label 0 = Original, 1 = Deepfake)
class CustomFaceDataset(Dataset):
    def __init__(self, root_dir, label, transform_rgb, transform_dwt):
        self.root_dir = root_dir
        self.transform_rgb = transform_rgb
        self.transform_dwt = transform_dwt
        self.images = [f for f in os.listdir(root_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
        self.label = label

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.images[idx])
        img = Image.open(img_path).convert('RGB')
        
        # RGB tensor
        rgb_tensor = self.transform_rgb(img)
        
        # DWT tensor
        dwt_img = self.dwt_features(img)
        dwt_tensor = self.transform_dwt(dwt_img)
        
        return rgb_tensor, dwt_tensor, self.label

    def dwt_features(self, img, wavelet='db1'):
        img = np.array(img)
        dwt_channels = []
        for ch in range(3):
            cA, _ = pywt.dwt2(img[..., ch], wavelet)
            cA_resized = Image.fromarray(cA).resize(img.shape[:2][::-1], Image.BILINEAR)
            dwt_channels.append(np.array(cA_resized))
        dwt_img = np.stack(dwt_channels, axis=2)
        return Image.fromarray(dwt_img.astype(np.uint8))

# TorchVision transforms (avoids scipy, compatible with Kaggle Python 3.11)
test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5]),
])


In [4]:
ff_deepfake_dir = "deeplearning/Deepfake/extracted_faces"
ff_original_dir = "deeplearning/Original/extracted_faces"

deepfake_dataset = CustomFaceDataset(ff_deepfake_dir, 1, test_transform, test_transform)
original_dataset = CustomFaceDataset(ff_original_dir, 0, test_transform, test_transform)
ff_test_dataset = ConcatDataset([deepfake_dataset, original_dataset])
ff_test_loader = DataLoader(ff_test_dataset, batch_size=32, shuffle=False)


In [5]:
class SimpleTwoBranchNet(nn.Module):
    def __init__(self, n_classes=2):
        super().__init__()
        self.rgb_backbone = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
        self.rgb_backbone.classifier = nn.Identity()
        self.dwt_backbone = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
        self.dwt_backbone.classifier = nn.Identity()
        self.classifier = nn.Sequential(
            nn.Linear(1280*2, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, n_classes)
        )
    def forward(self, rgb, dwt):
        f1 = F.adaptive_avg_pool2d(self.rgb_backbone.features(rgb), 1).view(rgb.size(0), -1)
        f2 = F.adaptive_avg_pool2d(self.dwt_backbone.features(dwt), 1).view(dwt.size(0), -1)
        x = torch.cat([f1, f2], dim=1)
        return self.classifier(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleTwoBranchNet(n_classes=2).to(device)
model.load_state_dict(torch.load("/kaggle/input/best-model-path1/tensorflow2/default/1/best_model.pth", map_location=device))
model.eval()


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/best-model-path1/tensorflow2/default/1/best_model.pth'

In [ ]:
import torch

# Your SimpleTwoBranchNet definition here (same as before)
# ...

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleTwoBranchNet(n_classes=2).to(device)

# Load the raw checkpoint
checkpoint = torch.load("/kaggle/input/best-model-path1/tensorflow2/default/1/best_model.pth", map_location=device)

# If checkpoint is a dict with 'state_dict' key (common), get it
if 'state_dict' in checkpoint:
    state_dict = checkpoint['state_dict']
else:
    state_dict = checkpoint

# Prepare a new dict filtering keys present in model
model_state_dict = model.state_dict()
filtered_dict = {}

for k, v in state_dict.items():
    # Remove unexpected prefix if exists like 'backbone.'
    key = k.replace('backbone.', '').replace('module.', '')  # Add more cleanup if needed

    if key in model_state_dict and v.shape == model_state_dict[key].shape:
        filtered_dict[key] = v
    else:
        print(f"Skipping weight {key} due to shape mismatch or missing in model")

# Load filtered weights with strict=False
model.load_state_dict(filtered_dict, strict=False)

print("Model weights loaded with filtered keys. Missing or mismatched keys were skipped safely.")


In [ ]:
from torch.utils.data import DataLoader, random_split, ConcatDataset
from torchvision import transforms
from PIL import Image
import numpy as np
import pywt
import os
import torch
from torch.utils.data import Dataset

# Dataset class (TorchVision transforms only, no albumentations)
class CustomFaceDataset(Dataset):
    def __init__(self, root_dir, label, transform_rgb, transform_dwt):
        self.root_dir = root_dir
        self.transform_rgb = transform_rgb
        self.transform_dwt = transform_dwt
        self.images = [f for f in os.listdir(root_dir) if f.lower().endswith(('.jpg','.png','.jpeg'))]
        self.label = label

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        path = os.path.join(self.root_dir, self.images[idx])
        img = Image.open(path).convert('RGB')
        rgb = self.transform_rgb(img)
        dwt_img = self.dwt_features(img)
        dwt = self.transform_dwt(dwt_img)
        return rgb, dwt, self.label

    def dwt_features(self, img, wavelet='db1'):
        img_np = np.array(img)
        dwt_c = []
        for ch in range(3):
            cA, _ = pywt.dwt2(img_np[..., ch], wavelet)
            cA = Image.fromarray(cA).resize(img_np.shape[:2][::-1], Image.BILINEAR)
            dwt_c.append(np.array(cA))
        dwt_img = np.stack(dwt_c, axis=2)
        return Image.fromarray(dwt_img.astype(np.uint8))

# TorchVision transforms
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.3, 0.3, 0.3, 0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])
val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Dataset paths
deepfake_dir = "/kaggle/input/ff-data/Deepfake/extracted_faces"
original_dir = "/kaggle/input/ff-data/Original/extracted_faces"

deepfake_ds = CustomFaceDataset(deepfake_dir, 1, train_transform, train_transform)
original_ds = CustomFaceDataset(original_dir, 0, train_transform, train_transform)

full_ds = ConcatDataset([deepfake_ds, original_ds])

# Split 80% train, 20% val
train_size = int(0.8 * len(full_ds))
val_size = len(full_ds) - train_size
train_ds, val_ds = random_split(full_ds, [train_size, val_size], generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2)

print(f"Train dataset size: {len(train_ds)}")
print(f"Validation dataset size: {len(val_ds)}")


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming train_loader and val_loader are already defined with your FF++ train/val splits

# Optimizer, criterion, scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.5, patience=2)

num_epochs = 10  # Adjust as needed
best_val_acc = 0
best_model_wts = model.state_dict()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    start = time.time()
    for rgb, dwt, labels in train_loader:
        rgb = rgb.to(device)
        dwt = dwt.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(rgb, dwt)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * labels.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total

    # Validation phase
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for rgb, dwt, labels in val_loader:
            rgb = rgb.to(device)
            dwt = dwt.to(device)
            labels = labels.to(device)
            outputs = model(rgb, dwt)
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)
    val_acc = val_correct / val_total
    scheduler.step(val_acc)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_wts = model.state_dict()
        torch.save(best_model_wts, "best_model_ffplus.pth")
        print(f"Validation accuracy improved to {best_val_acc:.4f}. Saved best model.")

    duration = time.time() - start
    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}, Time={duration:.1f}s")

# Load best weights for final evaluation
model.load_state_dict(best_model_wts)
model.eval()

# Final evaluation on validation or test set with confusion matrix and accuracy

all_preds = []
all_labels = []

with torch.no_grad():
    for rgb, dwt, labels in val_loader:
        rgb = rgb.to(device)
        dwt = dwt.to(device)
        labels = labels.to(device)
        outputs = model(rgb, dwt)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
print(f"Final Evaluation Accuracy: {accuracy*100:.2f}%")

# Plot confusion matrix
cm = np.zeros((2, 2), dtype=int)
for t, p in zip(all_labels, all_preds):
    cm[t, p] += 1

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Original', 'Deepfake'], yticklabels=['Original', 'Deepfake'])
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix on FF++ Dataset')
plt.show()
